# Homework 9
12110418 
庄子鲲
## Problem 1
Using Python and Numpy, write a class named SVMClassifier, which implements the SVM algorithm having slack variables and kernels such as Polynomial,Gaussian, and Sigmoid (using cvxopt package to solve the quadratic programing problem for Lagrange multipliers).

In [ ]:
import numpy as np
from cvxopt import matrix, solvers

class SVMClassifier:
    def __init__(self, kernel='linear', degree=3, gamma=None, coef0=0.0, C=1.0):
        self.kernel = kernel
        self.degree = degree
        self.gamma = gamma
        self.coef0 = coef0
        self.C = C
        self.alpha = None
        self.support_vectors = None
        self.support_vector_labels = None
        self.bias = None

    def fit(self, X, y):
        if self.kernel == 'linear':
            self.K = np.dot(X, X.T)
        elif self.kernel == 'poly':
            self.K = (np.dot(X, X.T) + self.coef0)**self.degree
        elif self.kernel == 'rbf':
            if self.gamma is None:
                self.gamma = 1.0 / X.shape[1]  # Default gamma
            self.K = np.exp(-self.gamma * ((X[:, np.newaxis] - X)**2).sum(axis=2))
        elif self.kernel == 'sigmoid':
            self.K = np.tanh(self.gamma * np.dot(X, X.T) + self.coef0)

        n_samples, n_features = X.shape
        P = matrix(np.outer(y, y) * self.K, tc='d')
        q = matrix(-np.ones(n_samples), tc='d')
        G = matrix(np.vstack((np.eye(n_samples), -np.eye(n_samples))), tc='d')
        h = matrix(np.hstack((self.C * np.ones(n_samples), np.zeros(n_samples))), tc='d')
        A = matrix(y.reshape(1, -1), tc='d')
        b = matrix(0.0, tc='d')

        # Solve the quadratic programming problem
        solution = solvers.qp(P, q, G, h, A, b)

        # Extract Lagrange multipliers from the solution
        self.alpha = np.array(solution['x']).flatten()

        # Support vectors have non-zero Lagrange multipliers
        sv_indices = np.where(self.alpha > 1e-5)[0]
        self.support_vectors = X[sv_indices]
        self.support_vector_labels = y[sv_indices]

        # Compute the bias term
        if self.kernel == 'linear':
            self.bias = np.mean(self.support_vector_labels - np.dot(self.alpha * self.support_vector_labels, self.K[sv_indices]))
        else:
            self.bias = 0  # Bias term not implemented for non-linear kernels

    def predict(self, X):
        if self.kernel == 'linear':
            decision_function = np.dot(self.alpha * self.support_vector_labels, np.dot(X, self.support_vectors.T)) + self.bias
        elif self.kernel == 'poly':
            decision_function = np.dot(self.alpha * self.support_vector_labels, (np.dot(X, self.support_vectors.T) + self.coef0)**self.degree) + self.bias
        elif self.kernel == 'rbf':
            decision_function = np.dot(self.alpha * self.support_vector_labels, np.exp(-self.gamma * ((X[:, np.newaxis] - self.support_vectors)**2).sum(axis=2))) + self.bias
        elif self.kernel == 'sigmoid':
            decision_function = np.dot(self.alpha * self.support_vector_labels, np.tanh(self.gamma * np.dot(X, self.support_vectors.T) + self.coef0)) + self.bias

        # Predict using the sign of the decision function
        return np.sign(decision_function)

# Example usage:
# Assuming X_train, y_train, X_test are your training features, training labels, and testing features
# svm = SVMClassifier(kernel='rbf', gamma=0.1, C=1.0)
# svm.fit(X_train, y_train)
# predictions = svm.predict(X_test)